In [1]:
model_name = "multi-qa-distilbert-cos-v1"

In [4]:
from tqdm.autonotebook import tqdm, trange

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

In [5]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
user_question_embedding = embedding_model.encode(user_question)

print(user_question_embedding[0])

0.078222655


In [7]:
import requests

def load_documents():
    base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
    relative_url = '03-vector-search/eval/documents-with-ids.json'
    docs_url = f'{base_url}/{relative_url}?raw=1'
    docs_response = requests.get(docs_url)
    documents = docs_response.json()
    return documents

In [8]:
docs = load_documents()

In [9]:
print(len(docs))

948


In [10]:
docs[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [11]:
print(type(docs))

<class 'list'>


In [15]:
course_name = "machine-learning-zoomcamp"
data = []
for doc in docs:
    if doc["course"] == course_name:
        data.append(doc)
print(len(data))

375


In [17]:
embeddings = []

for doc in data:
    question = doc["question"]
    text = doc["text"]
    qa_text = f'{question} {text}'
    qa_embedding = embedding_model.encode(qa_text)
    embeddings.append(qa_embedding)
    
print(len(embeddings))

375


In [19]:
import numpy as np

X = np.array(embeddings)
print(X.shape)

(375, 768)


In [20]:
print(len(user_question_embedding))

768


In [21]:
v = np.array(user_question_embedding)
print(v.shape)

(768,)


In [25]:
scores = X.dot(v)

In [26]:
print(type(scores))

<class 'numpy.ndarray'>


In [29]:
max_value = np.max(scores)
print(max_value)

0.6506573


In [30]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [31]:
search_engine = VectorSearchEngine(documents=data, embeddings=X)

In [32]:
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [33]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [38]:
print(type(ground_truth))
print(ground_truth[0])

<class 'list'>
{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}


In [56]:
sum = 0
for i in range(len(ground_truth)):
    question = ground_truth[i]['question']
    doc_id = ground_truth[i]['document']
    # print(question, doc_id)
    question_embedding = embedding_model.encode(question)
    results = search_engine.search(question_embedding, num_results=5)
    for hit in results:
        if hit['id'] == doc_id:
            sum += 1
            break

In [57]:
hit_rate = float(sum/len(ground_truth))

In [61]:
print(sum)
print(len(ground_truth))
print(hit_rate)

1720
1830
0.9398907103825137


In [62]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7b88509142b0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm 

In [ ]:
for doc in tqdm(embeddings):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)